In [ ]:
#⚡⚡ The requirements.txt file is a file where you can specify all the dependencies 
# (the installed packages that your project depends on) and their versions. This means 
# that you can share your project without all the installed packages, making it a lot 
# more lightweight. When someone downloads your project (like you have done here), the 
# requirements.txt file tells their code editor which packages need to be installed. 
#https://docs.google.com/document/d/e/2PACX-1vRIW_TuZ6z0ASjAoxgJgmzjGYLCDx019tKvphaTwK_Za7fnMKywUuXI0-s5wr0nQI_gprm6J6y7L9rL/pub

In [ ]:
#⚡⚡ Flask-Bootstrap Supports WTForms
# Literally, in 1 line of code, you can create your form. It's as simple as:
# {‌{ wtf.quick_form(form) }}
# What this line of code will do is generate all the labels, inputs, buttons, styling 
# for your form just by taking the WTForm object that was passed to the template (form).

In [ ]:
#⚡⚡ Create database with sqlite3.
import sqlite3

db = sqlite3.connect("books-collection.db")

cursor = db.cursor()

# create a new table (do it once).
# cursor.execute("CREATE TABLE books (id INTEGER PRIMARY KEY, title varchar(250) NOT NULL UNIQUE, author varchar(250) NOT NULL, rating FLOAT NOT NULL)")

# add new line to db (close the database otherwise, you'll get sqlite3.OperationalError: table books already exists)
cursor.execute("INSERT INTO books VALUES(1, 'Harry Potter', 'J. K. Rowling', '9.3')")
db.commit()

In [ ]:
#⚡⚡ Create database with flask.
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

##CREATE DATABASE
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///new-books-collection.db"
#Optional: But it will silence the deprecation warning in the console.
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

##CREATE TABLE
class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(250), unique=True, nullable=False)
    author = db.Column(db.String(250), nullable=False)
    rating = db.Column(db.Float, nullable=False)

    #Optional: this will allow each book object to be identified by its title when printed.
    def __repr__(self):
        return f'<Book {self.title}>'
    
db.create_all()

#CREATE RECORD
new_book = Book(id=1, title="Harry Potter", author="J. K. Rowling", rating=9.3)
db.session.add(new_book)
db.session.commit()

In [ ]:
# Create A New Record
new_book = Book(id=1, title="Harry Potter", author="J. K. Rowling", rating=9.3)
db.session.add(new_book)
db.session.commit()
# NOTE: When creating new records, the primary key fields is optional. you can also write:
new_book = Book(title="Harry Potter", author="J. K. Rowling", rating=9.3)
# the id field will be auto-generated.

# Read All Records
all_books = db.session.query(Book).all()

# Read A Particular Record By Query
book = Book.query.filter_by(title="Harry Potter").first()

# Update A Particular Record By Query
book_to_update = Book.query.filter_by(title="Harry Potter").first()
book_to_update.title = "Harry Potter and the Chamber of Secrets"
db.session.commit()  

# Update A Record By PRIMARY KEY
book_id = 1
book_to_update = Book.query.get(book_id)
book_to_update.title = "Harry Potter and the Goblet of Fire"
db.session.commit()  

# Delete A Particular Record By PRIMARY KEY
book_id = 1
book_to_delete = Book.query.get(book_id)
db.session.delete(book_to_delete)
db.session.commit()
# You can also delete by querying for a particular value e.g. by title or one of the other properties.

In [ ]:
# 🚨 Top Movies project main.
from flask import Flask, render_template, redirect, url_for, request
from flask_bootstrap import Bootstrap
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired
import requests
import random

app = Flask(__name__)
app.config['SECRET_KEY'] = '8BYkEfBA6O6donzWlSihBXox7C0sKR6b'
Bootstrap(app)

##CREATE DATABASE
app.config['SQLALCHEMY_DATABASE_URI'] = "sqlite:///top-movies-collection.db"
#Optional: But it will silence the deprecation warning in the console.
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

##CREATE TABLE
class Movie(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(250), unique=False, nullable=False)
    year = db.Column(db.String(15), nullable=False)
    description = db.Column(db.String(5000), nullable=False)
    rating = db.Column(db.Float, nullable=True)
    ranking = db.Column(db.Integer, nullable=True)
    review = db.Column(db.String(250), nullable=True, unique=False)
    img_url = db.Column(db.String(250), nullable=False)

db.create_all()

class MyDataRequired(DataRequired):
    field_flags = ()

class EditForm(FlaskForm):
    rating = StringField('Your rating e.g.7.5/10', validators=[MyDataRequired()])
    review = StringField('Your review', validators=[MyDataRequired()])
    submit = SubmitField('Done')

class AddForm(FlaskForm):
    m_title = StringField('Movie Title', validators=[MyDataRequired()])
    submit = SubmitField('Done')

@app.route("/")
def home():
    all_movies = Movie.query.order_by(Movie.rating).all()
    all_movies.reverse()
    for i in range(len(all_movies)):
        #This line gives each movie a new ranking reversed from their order in all_movies
        all_movies[i].ranking = i+1
    db.session.commit()
    return render_template("index.html", movies=all_movies)

@app.route('/edit', methods=['POST', 'GET'])
def edit():
    edit_form = EditForm()
    movie_id = request.args.get("id")
    movie_to_update = Movie.query.get(movie_id)
    if edit_form.validate_on_submit():
        movie_to_update.rating = float(edit_form.rating.data)
        movie_to_update.review = edit_form.review.data
        db.session.commit() 
        return redirect(url_for('home'))
    return render_template('edit.html', form=edit_form)

@app.route('/delete/')
def delete():
    movie_id = request.args.get("id")
    movie_to_delete = Movie.query.get(movie_id)
    db.session.delete(movie_to_delete)
    db.session.commit()
    return redirect(url_for('home'))

@app.route("/add", methods=["GET", "POST"])
def add():
    add_form = AddForm()
    if add_form.validate_on_submit():
        parameters = {
            'page': 1,
            'language': 'en-US',
            'api_key': '65935e0240814d9b0186b1b388628598',
            'include_adult': False,
            'query': add_form.m_title.data
        }
        response = requests.get(url='https://api.themoviedb.org/3/search/movie', params=parameters)
        response.raise_for_status()
        movies = response.json()
        return render_template('select.html', movies=movies)
    return render_template('add.html', form=add_form)

@app.route('/select')
def selected_movie():
    movie_id = request.args.get("m_id")
    response = requests.get(url=f'https://api.themoviedb.org/3/movie/{movie_id}?api_key=65935e0240814d9b0186b1b388628598')
    response.raise_for_status()
    movie = response.json()
    new_movie = Movie(
        title=movie["original_title"],
        year=movie["release_date"][:4],
        description=movie["overview"],
        rating=float(movie["vote_average"]),
        ranking = 0,
        img_url=f'https://image.tmdb.org/t/p/w500{movie["poster_path"]}',
        review= f'None{random.randint(0, 100)}',
    )
    db.session.add(new_movie)
    db.session.commit()
    return redirect(url_for('home'))

if __name__ == '__main__':
    app.run(debug=True)
